In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading dataset
df=pd.read_csv('RDKit.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 224 entries, Cate_name to Yield
dtypes: float64(177), int64(45), object(2)
memory usage: 119.1+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MaxAbsEStateIndex  Cate_MaxEStateIndex  Cate_MinAbsEStateIndex  \
0                8.669259             8.669259                0.076389   
1                8.669259             8.669259                0.076389   
2                8.669259             8.669259                0.076389   
3                8.669259             8.669259                0.076389   
4                8.850093             8.850093                0.060185   

   Cate_MinEStateIndex  Cate_qed  Cate_SPS  Cate_MolWt  Cate_HeavyAtomMolWt  \
0            -0.076389  0.490728  9.000000     110.112              104.064   
1            -0.076389  0.490728  9.000000     110.112              104.064   
2            -0.076389  0.490728  9.000000     110.112              104.064   
3            -0.076389  0.490728  9.000000     110.112              104.064   
4            -0.068889  0.513122  9.444444     124.139              116.075   

   Cate_ExactMolWt  Cate_NumValenceElectrons  ...  P_NumAromat

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
cum_coef = np.zeros((1,X.shape[1]))

#Lasso
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train = a_X_train.dropna(how='any', axis=1)
    a_X_test = a_X_test[a_X_train.columns]
    
    n_lambda = 1000
    alphas = np.logspace(-6, 0, n_lambda)
    model = LassoCV(cv=5, alphas=alphas)
    model.fit(a_X_train, y_train['Yield'])
    
    y_pred1 = model.predict(a_X_train)
    y_pred2 = model.predict(a_X_test)
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    print('----------------------')
    print('seed:', seed)
    print('R2_train:', r2_train)
    print('R2_test:', r2_test)
    print('----------------------')
print('R2_train_means:', sum(data_r2_train)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
R2_train: 0.900588411601112
R2_test: 0.6724934791527615
----------------------
----------------------
seed: 1
R2_train: 0.8475924005028652
R2_test: 0.752938775539911
----------------------
----------------------
seed: 2
R2_train: 0.8658112139435582
R2_test: 0.7689899551823135
----------------------
----------------------
seed: 3
R2_train: 0.8673897280108362
R2_test: 0.7774901199085236
----------------------
----------------------
seed: 4
R2_train: 0.8652329187502357
R2_test: 0.7595801040832095
----------------------
----------------------
seed: 5
R2_train: 0.8672005566634025
R2_test: 0.7176021484602606
----------------------
----------------------
seed: 6
R2_train: 0.8821270084491553
R2_test: 0.7131443030711112
----------------------
----------------------
seed: 7
R2_train: 0.876304854461071
R2_test: 0.7354135895043807
----------------------
----------------------
seed: 8
R2_train: 0.8836190709638209
R2_test: 0.6856109820533574
----------------------
----

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/RDKit/Lasso_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test
0     0.900588    8.561119   5.512090  0.672493  20.696913  13.346704
1     0.847592   11.739832   7.915633  0.752939  15.471419  11.614473
2     0.865811   11.141720   7.536938  0.768990  14.946600   9.413503
3     0.867390   11.121467   7.592790  0.777490  14.613452   9.831830
4     0.865233   10.778788   7.494953  0.759580  15.988881   9.990323
5     0.867201   10.369802   6.355439  0.717602  18.532496  11.617406
6     0.882127    9.835408   6.433505  0.713144  18.580082  12.290656
7     0.876305   10.877231   7.668458  0.735414  15.487081  11.203199
8     0.883619   10.120712   6.720152  0.685611  17.626415  12.750819
9     0.859359   11.938678   8.619676  0.529484  18.208594  12.725134
mean  0.871522   10.648476   7.184963  0.711275  17.015193  11.478405
